# All of Us Dataset Practice

In [1]:
import pandas as pd

### Patient profile

In [144]:
profile = pd.read_csv('D:/Boss/AllOfUs/df_A.csv', parse_dates=['date_of_birth'])

In [24]:
profile.gender.value_counts()

gender
Female                                                            245601
Male                                                              152142
Not man only, not woman only, prefer not to answer, or skipped     12521
No matching concept                                                   97
Name: count, dtype: int64

In [26]:
profile.race.value_counts()

race
White                        226615
Black or African American     78321
None Indicated                64614
Asian                         14162
PMI: Skip                      9054
More than one population       7720
None of these                  4351
Another single population      2987
I prefer not to answer         2537
Name: count, dtype: int64

In [29]:
profile.ethnicity.value_counts()

ethnicity
Not Hispanic or Latino                               320377
Hispanic or Latino                                    73945
PMI: Skip                                              9054
What Race Ethnicity: Race Ethnicity None Of These      4351
PMI: Prefer Not To Answer                              2537
No matching concept                                      97
Name: count, dtype: int64

### Patient with Some Types of Liver Cancer

In [18]:
hasLiverCancer = pd.read_csv('D:/Boss/AllOfUs/df_B.csv')

In [30]:
print('Number of Conditions')
hasLiverCancer.standard_concept_name.value_counts()

Number of Conditions


standard_concept_name
Secondary malignant neoplasm of liver                               33712
Liver cell carcinoma                                                16810
Primary malignant neoplasm of liver                                  8181
Intrahepatic bile duct carcinoma                                     7526
Secondary malignant neoplasm of liver and intrahepatic bile duct     3429
Malignant neoplasm of liver                                          2113
Primary malignant neoplasm of intrahepatic bile duct                 2023
Malignant neoplasm of intrahepatic gall duct                           73
Metastasis to liver from adenocarcinoma                                45
Malignant neoplasm of liver and intrahepatic bile ducts                23
Sarcoma of liver                                                        6
Adenocarcinoma of liver                                                 6
Hepatoblastoma                                                          4
Combined hepatoc

In [79]:
def get_earliest_date_lcancer(dataframe: pd.DataFrame= None):
    '''
    params: 
    accepts dataframe and get earliest date based on person id.
    The person with more than 2 conditions, will get the only earliest date 
    of the condition. We are not interested in what is the name of the condition.

    returns: dataframe consists of person id and earliest date
    '''
    output = {'person_id':[], 'csdLC':[]}
    dataframe['csd'] = dataframe.condition_start_datetime.apply(lambda x: pd.to_datetime(x, utc=True))
    ids = dataframe.person_id.unique()
    for id in ids:
        df = dataframe[dataframe.person_id == id]
        output['person_id'].append(id)
        output['csdLC'].append(df.csd.min())
    return output

In [80]:
hasLC = pd.DataFrame(get_earliest_date_lcancer(dataframe=hasLiverCancer))

In [84]:
hasLC.columns = ['person_id', 'csdLC']

### Disease Data

In [6]:
df = pd.read_csv('D:/Boss/AllOfUs/diseases.csv')

In [11]:
df2 = df[['person_id', 'scn']]
one_hot_encoded = pd.get_dummies(df2, columns=['scn'], prefix='', prefix_sep='')

In [32]:
disease_table = one_hot_encoded.groupby('person_id').max().reset_index(drop=False)

In [47]:
def get_earliest_date(dataframe: pd.DataFrame= None):
    '''
    params: 
    accepts dataframe and get earliest date based on person id.
    The person with more than 2 conditions, will get the only earliest date 
    of the condition. We are not interested in what is the name of the condition.

    returns: dataframe consists of person id and earliest date
    '''
    output = {'person_id':[], 'csd':[]}
    dataframe['csd'] = pd.to_datetime(dataframe.csd, format='%Y-%m-%d')
    ids = dataframe.person_id.unique()
    for id in ids:
        df = dataframe[dataframe.person_id == id]
        output['person_id'].append(id)
        output['csd'].append(df.csd.min())
    return output

In [48]:
# apply the function to dataframe
uniq_id_earliest_date = pd.DataFrame(get_earliest_date(dataframe=df))

In [55]:
# disease transformed to one hot encoding
disease_wider = pd.merge(disease_table, uniq_id_earliest_date, on='person_id', how='left')

In [68]:
disease_wider

,person_id,breast,cardiovascular,digestive,endocrine,genitourinary,hematopoietic,immune,integument,mediastinum,musculoskeletal,nervous,respiratory,csd
0,1000045,True,False,False,False,False,False,False,False,False,False,False,False,2012-05-01
1,1000091,True,False,False,False,False,False,False,False,False,False,False,False,2016-04-05
2,1000093,False,False,True,False,False,False,False,False,False,False,False,False,2020-07-23
3,1000095,False,False,True,False,False,False,False,False,False,False,False,True,2018-02-11
4,1000104,True,False,True,False,False,False,False,False,False,False,False,False,2017-08-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36035,9995183,False,False,True,False,False,False,False,False,False,False,False,False,2018-04-09
36036,9995407,True,False,False,False,False,False,False,False,False,False,False,False,2021-07-09
36037,9999318,False,False,False,True,False,False,False,False,False,False,False,False,2020-01-03
36038,9999351,False,False,False,False,False,False,False,False,False,True,False,False,2021-08-27


## Combine all tables

In [126]:
c1 = pd.merge(disease_wider, profile, on='person_id', how='outer')

In [146]:
final_table = pd.merge(c1, hasLC, on='person_id', how='outer')

In [151]:
#final_table.to_csv('D:/Boss/AllOfUs/final_table.csv', index=False)

In [152]:
pd.read_pickle('D:/Boss/AllOfUs/final_table.pkl')

,person_id,breast,cardiovascular,digestive,endocrine,genitourinary,hematopoietic,immune,integument,mediastinum,musculoskeletal,nervous,respiratory,csd,gender,date_of_birth,race,ethnicity,sex_at_birth,csdLC
0,1000045.0,True,False,False,False,False,False,False,False,False,False,False,False,2012-05-01,Female,1958-04-09 00:00:00+00:00,Asian,Not Hispanic or Latino,Female,NaT
1,1000091.0,True,False,False,False,False,False,False,False,False,False,False,False,2016-04-05,Female,1953-12-24 00:00:00+00:00,White,Not Hispanic or Latino,Female,NaT
2,1000093.0,False,False,True,False,False,False,False,False,False,False,False,False,2020-07-23,Female,2000-01-13 00:00:00+00:00,White,Not Hispanic or Latino,Female,NaT
3,1000095.0,False,False,True,False,False,False,False,False,False,False,False,True,2018-02-11,Female,1978-03-25 00:00:00+00:00,White,Not Hispanic or Latino,Female,NaT
4,1000104.0,True,False,True,False,False,False,False,False,False,False,False,False,2017-08-20,Female,1964-08-29 00:00:00+00:00,None Indicated,Hispanic or Latino,Female,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410356,2524505.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Female,1957-01-11 00:00:00+00:00,White,Not Hispanic or Latino,Female,NaT
410357,3452399.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Female,1946-02-20 00:00:00+00:00,White,Not Hispanic or Latino,Female,NaT
410358,2155804.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Female,1954-11-15 00:00:00+00:00,White,Not Hispanic or Latino,Female,NaT
410359,5943794.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Female,1984-03-05 00:00:00+00:00,White,Not Hispanic or Latino,Female,NaT


### Disease dataset: get earliest date

In [2]:
# read disease and save 
disease = pd.read_csv('D:/F2023/AllOfUs/df_C.csv')

In [3]:
disease['csd'] = disease['condition_start_datetime'].apply(lambda x: pd.to_datetime(x))

In [4]:
def simpler_concept(condition: str=None):
    '''
    params:
    accepts string of standard concept name
    return:
    shortened version of standard concept name
    '''
    new_str = ''
    if condition == 'Disorder of breast':
       new_str = 'breast'
    elif condition == 'Disorder of digestive system':
       new_str = 'digestive'
    elif condition == 'Disorder of musculoskeletal system':
       new_str = 'musculoskeletal'
    elif condition == 'Disorder of hematopoietic structure':
       new_str = 'hematopoietic'
    elif condition == 'Disorder of respiratory system':
       new_str = 'respiratory'
    elif condition == 'Disorder of cardiovascular system':
       new_str = 'cardiovascular'
    elif condition == 'Disorder of endocrine system':
       new_str = 'endocrine'
    elif condition == 'Disorder of integument':
       new_str = 'integument'
    elif condition == 'Disorder of nervous system':
       new_str = 'nervous'
    elif condition == 'Disorder of the genitourinary system':
       new_str = 'genitourinary'
    elif condition == 'Disorder of mediastinum':
       new_str = 'mediastinum'
    elif condition == 'Disorder of immune structure':
       new_str = 'immune'
    else:
       print('condition is not shortened!!')
    return new_str

In [5]:
disease['scn'] = disease.standard_concept_name.apply(simpler_concept)

In [6]:
disease['csd2'] = disease.loc[:,'csd'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [7]:
disease2 = disease[['person_id','scn','standard_concept_code', 'csd2']].copy()

In [8]:
disease2.columns = ['person_id', 'scn', 'scc', 'csd2']
disease2

,person_id,scn,scc,csd2
0,1152610,cardiovascular,49601007,2018-08-29
1,2621724,cardiovascular,49601007,2012-02-28
2,1801041,cardiovascular,49601007,2019-05-18
3,1956972,cardiovascular,49601007,2018-09-24
4,1529466,cardiovascular,49601007,2022-02-12
...,...,...,...,...
147359,1879747,nervous,118940003,2019-12-11
147360,1700098,nervous,118940003,2012-07-17
147361,3248701,nervous,118940003,2018-10-01
147362,2601573,nervous,118940003,2020-02-02


In [9]:
import numpy as np
pid = np.array(disease2.person_id.unique())

In [83]:
pd.reset_option('display.max_rows')

In [10]:
#patient with number of conditions
patient_nth_condtions = disease2.groupby(['person_id','scn']).count()
patient_nth_condtions # saved in 'D:\\F2023\\AllOfUs\\patient_with_nth_conditions.csv'

scc  csd2
person_id scn                       
1000045   breast             2     2
1000091   breast             1     1
1000093   digestive          1     1
1000095   digestive          1     1
          respiratory        2     2
...                        ...   ...
9995183   digestive          1     1
9995407   breast             6     6
9999318   endocrine          3     3
9999351   musculoskeletal    1     1
9999653   cardiovascular     2     2

[46896 rows x 2 columns]

In [12]:
df1 = pd.read_csv('D:/F2023/AllOfUs/patient_with_nth_condtions.csv')

In [50]:
# patient with nth conditions
df1.person_id.value_counts().reset_index(name='count')#.to_csv('D:/F2023/AllOfUs/condition_count.csv', index=False)

In [27]:
nth_condition_wider = df1.pivot(index='person_id', columns='scn', values='scc').reset_index(drop=False)

In [52]:
#nth_condition_wider.fillna(0).to_csv('D:/F2023/AllOfUs/nth_condition_wider.csv', index=False)
nth_condition_wider = pd.read_csv('D:/F2023/AllOfUs/nth_condition_wider.csv')
nth_condition_wider

,person_id,breast,cardiovascular,digestive,endocrine,genitourinary,hematopoietic,immune,integument,mediastinum,musculoskeletal,nervous,respiratory
0,1000045,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000091,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000093,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000095,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,1000104,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36035,9995183,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36036,9995407,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36037,9999318,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36038,9999351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### TODO: get earliest date

In [60]:
def earliest_date(id: int=None, dataframe: pd.DataFrame=None):
    df = dataframe[dataframe['person_id'] == id]
    return df.sort_values(by='csd2')

earliest_date(id=pid[0], dataframe=disease2)


,person_id,scn,scc,csd2
24755,1152610,cardiovascular,49601007,2010-04-10
69287,1152610,cardiovascular,49601007,2010-05-25
83997,1152610,cardiovascular,49601007,2010-05-26
53794,1152610,cardiovascular,49601007,2010-06-25
39453,1152610,cardiovascular,49601007,2010-11-20
...,...,...,...,...
98350,1152610,cardiovascular,49601007,2021-09-02
112982,1152610,cardiovascular,49601007,2021-09-30
39405,1152610,cardiovascular,49601007,2021-11-04
98713,1152610,cardiovascular,49601007,2021-11-30


In [98]:
# patients with some type of liver conditions
#pd.read_csv('D:/F2023/AllOfUs/df_B.csv').groupby(['person_id', 'standard_concept_name']).count().to_csv('D:/F2023/AllOfUs/patient_with_sometype_livercondition.csv')
patients_wlcondition = pd.read_csv('D:/F2023/AllOfUs/patient_with_sometype_livercondition.csv')
patients_wlcondition

,person_id,standard_concept_name,standard_concept_code,condition_start_datetime
0,1002037,Intrahepatic bile duct carcinoma,1,1
1,1002037,Secondary malignant neoplasm of liver,3,3
2,1002156,Secondary malignant neoplasm of liver,2,2
3,1002627,Secondary malignant neoplasm of liver,4,4
4,1002962,Secondary malignant neoplasm of liver,89,89
...,...,...,...,...
3087,9898011,Liver cell carcinoma,23,23
3088,9898011,Primary malignant neoplasm of liver,24,24
3089,9967236,Secondary malignant neoplasm of liver,21,21
3090,9978622,Primary malignant neoplasm of liver,1,1


In [99]:
# with liver conditions
with_lcondition = pd.read_csv('D:/F2023/AllOfUs/df_B.csv')

In [103]:
with_lcondition['csd'] = with_lcondition.condition_start_datetime.apply(lambda x: pd.to_datetime(x))

In [115]:
with_lcondition['person_id'].value_counts()

person_id
5888053    955
1934045    877
1971463    744
1234266    729
1815398    709
          ... 
2002488      1
1395655      1
6315407      1
2293524      1
2704301      1
Name: count, Length: 2171, dtype: int64